In [1]:
import cv2
import mtcnn.mtcnn
import numpy as np
from PIL import Image
import mysql.connector
from pathlib import Path
import matplotlib.pyplot as plt 
from keras.models import load_model


Using TensorFlow backend.
/home/noor/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/noor/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/noor/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/noor/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [2]:
unique_id=input("Insert the Identity Number : ")

Insert the Identity Number : 420


In [3]:
#create database connection using myconnector
#connect database created in creat_db
face_db=mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aimfame1@",
        database="face_db")

In [4]:
#define mtcnn detector
detector=mtcnn.MTCNN()
#load pre-trained keras model
model=load_model("facenet_keras.h5")

Instructions for updating:
Colocations handled automatically by placer.


/home/noor/.local/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
def preprocess(face_img):
    #preprocess input image
    
    #model expects input image of size 160*160
    #convert numpy array to image type
    face_img=Image.fromarray(face_img)
    #resize image
    face_img=face_img.resize((160, 160))
    #revert to numpy array
    face_img=np.asarray(face_img)
    print(face_img.shape)
    #standardize the pixels of sample
    mean,std=face_img.mean(),face_img.std()
    print(mean,std)
    face_img=(face_img-mean)/std
    #add fourth dimension to numpy array(number of images)
    sample=np.expand_dims(face_img,axis=0)
    print(sample.shape)
    return sample

In [6]:
path="/home/noor/Desktop/bradpitt1.jpg"
img=plt.imread(path)
results = detector.detect_faces(img)

In [7]:
x1,y1,width,height=results[0]["box"]
#calculate (x2,y2) coordinate
x2,y2=x1+width,y1+height
cropped_img=img[y1:y2,x1:x2]
print(cropped_img.shape)

(739, 566, 3)


In [8]:
sample=preprocess(cropped_img)

(160, 160, 3)
138.27307291666668 56.504495709180965
(1, 160, 160, 3)


In [9]:
pred=model.predict(sample)
emb_vec=np.around(pred[0],decimals=3) 

In [10]:
#wrap numpy array into string format
string=','.join(str(x) for x in emb_vec)

In [ ]:
#capture video from camera
cap=cv2.VideoCapture(0)
while (cap.isOpened()):
    #capture frame by frame
    red,frame=cap.read()
    #detect the face using detector
    face=detector.detect_faces(frame)
    if len(face)==1 :
        x1,y1,width,height=face[0]["box"]
        x1,y1=abs(x1),abs(y1)
        #calculate (x2,y2) coordinate
        x2,y2=x1+width,y1+height
        face_img=image[y1:y2,x1:x2]
        sample=preprocess(face_img)
        #get embedding vector
        pred=model.predict(sample)
        emb_vec=pred[0]
        

In [11]:
#create cursor object
mycursor=face_db.cursor()

In [12]:
#insert row into the table
query="INSERT INTO Registered_users (UniqueID,Embedding_Vector) VALUES (%s,%s)"
mycursor.execute(query,(unique_id,string))
#commit changes
face_db.commit()

In [15]:
mycursor.close()
face_db.close()

In [13]:
#show all rows in the table
"""
mycursor.execute("SELECT * FROM Registered_users")
#fetch all rows from the above executed query
result=mycursor.fetchall()
for i in result:
    print(i)
"""    

(420, '0.101,-0.201,0.302,-0.694,-0.002,-1.581,-0.427,-0.042,0.995,-0.22,0.308,-1.182,0.724,-0.393,0.925,2.188,-1.262,0.627,0.114,0.357,2.071,0.774,0.448,0.681,0.902,0.68,-1.394,0.044,1.501,0.524,0.617,-0.837,0.594,1.311,-0.119,0.368,0.607,-0.827,0.57,-0.344,0.981,1.317,0.204,-2.292,-0.159,1.745,-0.676,-0.658,-0.476,0.217,-0.42,-0.218,-1.937,1.312,-1.188,-0.777,0.115,0.066,0.127,0.556,1.916,-0.193,0.949,0.262,-0.666,-1.122,0.666,-0.871,0.12,-0.217,0.782,-0.206,0.268,-1.267,0.641,-1.539,0.815,0.968,0.353,-0.208,-1.033,-1.31,0.921,-0.842,-1.496,0.26,-0.669,0.174,0.564,-0.921,1.279,0.703,0.881,0.915,0.361,-0.893,-0.398,-0.14,-0.686,-0.023,0.758,1.989,0.565,0.089,1.093,0.895,-1.068,-0.163,-2.44,1.735,0.013,-0.511,0.304,-0.9,1.136,-0.634,0.322,-0.797,-1.116,-1.01,-1.12,1.168,1.156,-0.582,0.147,-1.15,-0.675,0.71')
